In [105]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [106]:
filepath = "ratings.csv"

In [107]:
# Create a dataframe from csv; drop all rows with no movie title.
df_raw = pd.read_csv(filepath, delimiter=',')

In [108]:
ratings_df = df_raw[['movieId','rating']]
ratings_df

,movieId,rating
0,110,1.0
1,147,4.5
2,858,5.0
3,1221,5.0
4,1246,5.0
...,...,...
26024284,58559,5.0
26024285,60069,5.0
26024286,63082,4.5
26024287,64957,4.5


In [109]:
ratings_df['ratings_count'] = ratings_df.groupby('movieId')['rating'].size()
ratings_df

C:\Users\bryan\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,movieId,rating,ratings_count
0,110,1.0,NaN
1,147,4.5,66008.0
2,858,5.0,26060.0
3,1221,5.0,15497.0
4,1246,5.0,2981.0
...,...,...,...
26024284,58559,5.0,NaN
26024285,60069,5.0,NaN
26024286,63082,4.5,NaN
26024287,64957,4.5,NaN


In [110]:
ratings_df['avg_rating'] = ratings_df.groupby('movieId')['rating'].mean()
ratings_df


,movieId,rating,ratings_count,avg_rating
0,110,1.0,NaN,NaN
1,147,4.5,66008.0,3.888157
2,858,5.0,26060.0,3.236953
3,1221,5.0,15497.0,3.175550
4,1246,5.0,2981.0,2.875713
...,...,...,...,...
26024284,58559,5.0,NaN,NaN
26024285,60069,5.0,NaN,NaN
26024286,63082,4.5,NaN,NaN
26024287,64957,4.5,NaN,NaN


In [111]:
ratings_df['median_rating'] = ratings_df.groupby('movieId')['rating'].median()
ratings_df

,movieId,rating,ratings_count,avg_rating,median_rating
0,110,1.0,NaN,NaN,NaN
1,147,4.5,66008.0,3.888157,4.0
2,858,5.0,26060.0,3.236953,3.0
3,1221,5.0,15497.0,3.175550,3.0
4,1246,5.0,2981.0,2.875713,3.0
...,...,...,...,...,...
26024284,58559,5.0,NaN,NaN,NaN
26024285,60069,5.0,NaN,NaN,NaN
26024286,63082,4.5,NaN,NaN,NaN
26024287,64957,4.5,NaN,NaN,NaN


In [112]:
del ratings_df['rating']
ratings_df = ratings_df.drop_duplicates(subset='movieId', keep='first')
ratings_df = ratings_df.rename(columns={'movieId':'movie_id'})
ratings_df = ratings_df.dropna()
ratings_df

,movie_id,ratings_count,avg_rating,median_rating
1,147,66008.0,3.888157,4.00
2,858,26060.0,3.236953,3.00
3,1221,15497.0,3.175550,3.00
4,1246,2981.0,2.875713,3.00
5,1968,15258.0,3.079565,3.00
...,...,...,...,...
175437,166558,2.0,3.250000,3.25
175621,146036,2.0,3.000000,3.00
175879,6902,1.0,4.000000,4.00
176021,77596,1.0,1.500000,1.50


In [113]:
# Engine connection
rds_connection_string = "postgres:postgres@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [114]:
# Verifying that connection is made
engine.table_names()

['movies_data',
 'crew_movie_junction',
 'country_movie_junction',
 'production_countries',
 'company_movie_junction',
 'actors',
 'production_companies',
 'crew',
 'cast_movie_junction',
 'ratings']

In [116]:
ratings_df.to_sql(name='ratings', con=engine, if_exists='append', index=False)